In [2]:
from datetime import datetime, timedelta, timezone
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from meteostat import Point, Hourly, Daily, Stations
import sqlite3
from astropy.time import Time
import requests
import time
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import ee

In [3]:
# Load and preprocess Fire_Data
df = pd.read_csv('../Data/Fire_Data.csv')
df['incident_name'] = df['incident_name'].str.replace('Fire', '', regex=False).str.strip().str.lower()
df['incident_dateonly_created'] = pd.to_datetime(df['incident_dateonly_created'].astype(str).str.split("+").str[0])

# Load and preprocess California_Fire_Perimeters_(all)
df_cause = pd.read_csv("../Data/California_Fire_Perimeters_(all).csv")
df_cause['ALARM_DATE'] = pd.to_datetime(df_cause['ALARM_DATE'].astype(str).str.split("+").str[0])
df_cause['FIRE_NAME'] = df_cause['FIRE_NAME'].str.strip().str.lower()

# Add a constant key to allow cross join
df['merge_key'] = 1
df_cause['merge_key'] = 1

# Cross join
df_cross = pd.merge(df, df_cause, on='merge_key', suffixes=('', '_cause')).drop(columns=['merge_key'])

# Filter rows where date difference is within ±1 day
df_cross = df_cross[
    (df_cross['ALARM_DATE'] >= df_cross['incident_dateonly_created'] - timedelta(days=1)) &
    (df_cross['ALARM_DATE'] <= df_cross['incident_dateonly_created'] + timedelta(days=1))
]

# Drop rows where names are missing
df_cross = df_cross.dropna(subset=['incident_name', 'FIRE_NAME'])

# Filter rows where one name is contained in the other
df_cross = df_cross[
    df_cross.apply(
        lambda row: row['incident_name'] in row['FIRE_NAME'] or row['FIRE_NAME'] in row['incident_name'],
        axis=1
    )
]

# Final merged DataFrame
df_final = df_cross.copy()

# Display number of matched rows
print(f"Merged rows: {len(df_final)}")


Merged rows: 1657


In [4]:
df_final = df_final.reset_index()
df_final = df_final.drop(columns="index")
df_final.head()

,incident_name,incident_is_final,incident_date_last_update,incident_date_created,incident_administrative_unit,incident_administrative_unit_url,incident_county,incident_location,incident_acres_burned,incident_containment,...,Management Objective,GIS_ACRES,Comments,Complex Name,IRWIN ID,Fire Number (historical use),Complex ID,DECADES,Shape__Area,Shape__Length
0,bridge,Y,2018-01-09T13:46:00Z,2017-10-31T11:22:00Z,Shasta-Trinity National Forest,NaN,Shasta,"I-5 and Turntable Bay, 7 miles NE of Shasta Lake",37.0,100.0,...,1.0,35.51982,Started by burning semi trailer truck carrying...,NaN,{D32AB87C-D4D7-4573-834C-D07F81C69383},NaN,NaN,2010-2019,2.508758e+05,3192.924039
1,pala,Y,2020-09-16T14:07:35Z,2009-05-24T14:56:00Z,CAL FIRE San Diego Unit,NaN,San Diego,"Hwy 76 and Pala Temecula, northwest of Pala",122.0,100.0,...,1.0,106.11540,NaN,NaN,NaN,NaN,NaN,2000-2009,6.172894e+05,4424.336593
2,river,Y,2022-10-24T11:39:23Z,2013-02-24T08:16:00Z,CAL FIRE San Bernardino Unit,NaN,Inyo,"south of Narrow Gauge Rd & north of Hwy 136, e...",407.0,100.0,...,1.0,406.84150,Lone Pine Area,NaN,NaN,NaN,NaN,2010-2019,2.559475e+06,8138.932864
3,fawnskin,Y,2013-04-22T09:00:00Z,2013-04-20T17:30:00Z,San Bernardino National Forest,NaN,San Bernardino,"west of Delamar Mountain, north of the communi...",30.0,100.0,...,1.0,14.40060,NaN,NaN,NaN,NaN,NaN,2010-2019,8.557668e+04,2180.966397
4,gold,Y,2013-05-01T07:00:00Z,2013-04-30T12:59:00Z,CAL FIRE Madera-Mariposa-Merced Unit,NaN,Madera,Between Road 210 and Road 200 near Fine Gold C...,274.0,100.0,...,1.0,183.64240,NaN,NaN,NaN,NaN,NaN,2010-2019,1.170901e+06,4834.204082


In [4]:
#Retrieve Weather Data using open-meteo api
import time
url = "https://archive-api.open-meteo.com/v1/archive"

df = df_final

for index in df.index:
    try:
        lat = df.loc[index,'incident_latitude'] # Latitude
        long = df.loc[index,'incident_longitude'] # Longitude
        start = datetime.strptime(df.loc[index,'incident_date_created'].split("T")[0], "%Y-%m-%d") # Start date
        try :
            end = datetime.strptime(df.loc[index,'incident_date_extinguished'].split("T")[0], "%Y-%m-%d")
        except:
            end= datetime.now(timezone.utc)

        params = {
            "latitude": lat,  
            "longitude": long, 
            "start_date": start.strftime('%Y-%m-%d'), 
            "end_date": end.strftime('%Y-%m-%d'),  
            "hourly": [ "precipitation"],  # Get hourly temperature and precipitation "temperature_2m", , "windspeed_10m"
            "timezone": "UTC"  
        }
        time.sleep(1)
        # Make API request
        response = requests.get(url, params=params)

        # Convert response to JSON
        data = response.json()

        df.loc[index,'min_temp'] = min(data["hourly"]["temperature_2m"])
        df.loc[index,'max_temp'] = max(data["hourly"]["temperature_2m"])
        df.loc[index,'avg_temp'] = np.mean(data["hourly"]["temperature_2m"]) ## Unit: °C
        df.loc[index,'avg_windspeed'] = np.mean(data["hourly"]["windspeed_10m"]) ## Unit: km/h
        df.loc[index,'avg_precipitation'] = np.mean(data["hourly"]["precipitation"]) ## Unit: mm
    except:
        print(f'The index that failed is {index}')

In [78]:
#Add new column for the year the fire started
df['Year_Started'] = pd.to_datetime(df['incident_dateonly_created']).dt.year
df.loc[df['incident_acres_burned'].isna(),"Year_Started"].value_counts()
#Remove columns for which the values are missing
df.dropna(subset=['incident_acres_burned', 'min_temp', 'max_temp', 'avg_temp', 'avg_windspeed', 'avg_precipitation'], inplace=True)

In [6]:
data = pd.read_csv("../Data/WildFire_DataSet.csv")
data.head()

,incident_id,incident_url,incident_type,incident_name,incident_date_created,incident_date_extinguished,incident_date_last_update,incident_dateonly_extinguished,incident_dateonly_created,incident_is_final,...,ALARM_DATE,GIS_ACRES,CAUSE,Year_Started,ndvi,avg_pdsi,avg_spi30d,fuel_type,cause_description,ndmi
0,2ca11d45-8139-4c16-8af0-880d99b21e82,https://www.fire.ca.gov/incidents/2017/10/31/b...,NaN,bridge,2017-10-31 11:22:00+00:00,2018-01-09 13:46:00+00:00,2018-01-09T13:46:00Z,1/9/18,10/31/17,Y,...,10/31/17,35.51982,2,2017,0.871471,2.720000,0.951667,Forest,Equipment/vehicles,0.214126
1,8f61f461-552d-4538-b186-35ab030da416,https://www.fire.ca.gov/incidents/2009/5/24/pa...,Wildfire,pala,2009-05-24 14:56:00+00:00,2009-05-25 00:00:00+00:00,2020-09-16T14:07:35Z,5/25/09,5/24/09,Y,...,5/24/09,106.11540,14,2009,NaN,NaN,NaN,NaN,Unknown (Human),0.005950
2,094719ba-a47b-4abb-9ec5-a506b2b9fd23,https://www.fire.ca.gov/incidents/2013/2/24/ri...,NaN,river,2013-02-24 08:16:00+00:00,2013-02-28 20:00:00+00:00,2022-10-24T11:39:23Z,2/28/13,2/24/13,Y,...,2/24/13,406.84150,14,2013,0.088693,-2.080000,-0.218333,Shrubland,Unknown (Human),-0.038049
3,357ffc13-bef9-48eb-810f-c5de851972eb,https://www.fire.ca.gov/incidents/2013/4/30/go...,NaN,gold,2013-04-30 12:59:00+00:00,2013-05-01 07:00:00+00:00,2013-05-01T07:00:00Z,5/1/13,4/30/13,Y,...,4/30/13,183.64240,14,2013,0.021794,-0.891667,-0.225000,Grassland,Unknown (Human),0.048903
4,53122f0f-fefc-4dbf-b2d8-566b42ced66d,https://www.fire.ca.gov/incidents/2013/5/1/pan...,NaN,panther,2013-05-01 09:12:00+00:00,2013-05-09 09:00:00+00:00,2022-10-24T11:40:03Z,5/9/13,5/1/13,Y,...,5/1/13,6896.19800,9,2013,0.136799,1.003333,-0.231667,Forest,Miscellaneous/other,0.109611


In [83]:
# Map a function that returns a Feature (not a dictionary)
def extract_ndvi(image):
    mean_dict = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=250
    )
    feature = ee.Feature(None, {
        'NDVI': mean_dict.get('NDVI'),
        'date': image.date().format('YYYY-MM-dd')
    })
    return feature
for index in data.index:
    try:
        # Define your location
        lat = data.loc[index,'incident_latitude'] # Latitude
        long = data.loc[index,'incident_longitude'] 
        point = ee.Geometry.Point([long, lat])
        start = data.loc[index,'incident_dateonly_created'] # Start date
        try :
            end = data.loc[index,'incident_dateonly_extinguished']
        except:
            end= date.today()

        # Load MODIS NDVI ImageCollection
        collection = ee.ImageCollection('MODIS/006/MOD13Q1') \
            .filterBounds(point) \
            .filterDate(start, end) \
            .select('NDVI')

        # Apply map
        ndvi_features = collection.map(extract_ndvi)

        # Convert FeatureCollection to a list of dictionaries
        ndvi_list = ndvi_features.aggregate_array('NDVI').getInfo()
        data.loc[index,'avg_ndvi'] = np.mean(ndvi_list)
    except:
        print(f'The index that failed is {index}')

In [ ]:
#Retrive the drought Index pdsi, spi30d
def get_drought_index(latitude, longitude, start_date, end_date, drought_index_name='pdsi'):
    point = ee.Geometry.Point(longitude, latitude)
    drought = ee.ImageCollection('GRIDMET/DROUGHT') \
        .select(drought_index_name) \
        .filterDate(start_date, end_date)
    ts = drought.getRegion(point, 1).getInfo()

    if len(ts) > 1:
        data = [item[4] for item in ts[1:]]
        return data
    else:
        return None

for index in data.index:
    try:
        latitude = data.loc[index,'incident_latitude'] # Latitude
        longitude = data.loc[index,'incident_longitude'] 
        start_date = datetime(int(data.loc[index,'Year_Started']), 1, 1)
        end_date = start_date + timedelta(days=30)
        # Convert to string format for filtering
        start_date = start_date.strftime('%Y-%m-%d')
        end_date = end_date.strftime('%Y-%m-%d')
        drought_index = 'pdsi'  # You can change this to other indices
        drought_data = get_drought_index(latitude, longitude, start_date, end_date, drought_index)
        data.loc[index,'avg_pdsi'] = np.mean(drought_data)
        drought_index_spi = 'spi30d'
        spi_data = get_drought_index(latitude, longitude, start_date, end_date, drought_index_spi)
        data.loc[index,'avg_spi30d'] = np.mean(spi_data)
    except:
        print(f'The index that failed is {index}')

In [ ]:
#Retrive NDVI value
def get_current_ndvi(latitude, longitude,year):
    point = ee.Geometry.Point([longitude, latitude])
    
    # Set time window: last 30 days
    
    # Set start date to 30 days before
    start_date = datetime(int(year), 1, 1)
    end_date = start_date + timedelta(days=30)
    # Convert to string format for filtering
    start_date = start_date.strftime('%Y-%m-%d')
    end_date = end_date.strftime('%Y-%m-%d')
    # Load and filter Sentinel-2 imagery
    collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(point) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 100)) \
        .sort('system:time_start', False)

    # If the collection is empty, return None
    if collection.size().getInfo() == 0:
        print("No suitable Sentinel-2 image found for the given location and date range.")
        return None

    # Compute NDVI
    def add_ndvi(image):
        ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
        return image.addBands(ndvi)

    ndvi_collection = collection.map(add_ndvi)
    latest_image = ndvi_collection.first()

    # Get NDVI value at the point
    ndvi_value = latest_image.select('NDVI').reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=10
    ).get('NDVI')

    return ee.Number(ndvi_value).getInfo()
#data.index
for index in data.index:
    try:
        lat = data.loc[index,'incident_latitude'] # Latitude
        lon = data.loc[index,'incident_longitude']
        ndvi = get_current_ndvi(lat, lon, data.loc[index,'Year_Started'])
        data.loc[index,'ndvi'] = ndvi
    except:
        print(f'The index that failed is {index}')

In [ ]:
nlcd_to_fuel = {
    11: 'Non-burnable', 12: 'Non-burnable',
    21: 'Urban', 22: 'Urban', 23: 'Urban', 24: 'Urban',
    31: 'Barren',
    41: 'Forest', 42: 'Forest', 43: 'Forest',
    52: 'Shrubland',
    71: 'Grassland', 81: 'Grassland',
    82: 'Agriculture',
    90: 'Wetland', 95: 'Wetland'
}

# Map to new column
data['fuel_type'] = data['landcover'].map(nlcd_to_fuel)

In [ ]:
cause_map = {
    1: "Natural",
    2: "Equipment/vehicles",
    3: "Smoking",
    4: "Recreation/cultural activities",
    5: "Debris/open burning",
    6: "Railroad",
    7: "Incendiary (unlawful)",
    8: "Fire Play (minor)",
    9: "Miscellaneous/other",
    10: "Fireworks",
    11: "Power generation/transmission",
    12: "Structure",
    13: "Ammunition/explosives",
    14: "Unknown (Human)",
}


# Map to new column
data['cause_description'] = data['CAUSE'].map(cause_map).fillna('Unknown')

In [ ]:
# Group and summarize
fuel_stats = data.groupby('fuel_type')['incident_acres_burned'].mean().reset_index()

# Sort for better readability
fuel_stats = fuel_stats.sort_values(by='incident_acres_burned', ascending=False)

# Plot
plt.figure(figsize=(10, 6))
sns.barplot(data=fuel_stats, x='fuel_type', y='incident_acres_burned', palette='viridis')
plt.title('Average Acres Burned by Fuel Type')
plt.xlabel('Fuel Type')
plt.ylabel('Average Acres Burned')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize()

# Prepare feature lists
elevations, landcovers, aspects, slopes, ndmis,  = [], [], [], [], []

# Datasets
elevation_dataset = ee.Image('USGS/SRTMGL1_003')
landcover_dataset = ee.Image('USGS/NLCD_RELEASES/2019_REL/NLCD/2019').select('landcover')
terrain = ee.Terrain.products(elevation_dataset)

# Utility to extract value
def extract_mean_value(image, region, band, scale=30):
    try:
        sample = image.select(band).sample(region=region, scale=scale)
        if sample.size().getInfo() > 0:
            return sample.first().get(band).getInfo()
    except Exception as e:
        print(f"Error extracting {band}: {e}")
    return None


# Iterate through data
for idx, row in data.iterrows():
    lat, lon = row['incident_latitude'], row['incident_longitude']
    point = ee.Geometry.Point([lon, lat])
    buffered_point = point.buffer(30)

    # Parse incident date
    try:
        fire_date = datetime.strptime(row['incident_dateonly_created'], '%Y-%m-%d')
    except:
        fire_date = datetime(2020, 6, 1)  # fallback if date parsing fails

    year = fire_date.year
    date_start = fire_date.strftime('%Y-%m-%d')
    date_end = (fire_date.replace(month=fire_date.month % 12 + 1, day=1) if fire_date.month < 12 else fire_date.replace(year=year + 1, month=1, day=1)).strftime('%Y-%m-%d')

    # Extract features
    elevations.append(extract_mean_value(elevation_dataset, buffered_point, 'elevation'))
    landcovers.append(extract_mean_value(landcover_dataset, buffered_point, 'landcover'))
    aspects.append(extract_mean_value(terrain.select('aspect'), buffered_point, 'aspect'))
    slopes.append(extract_mean_value(terrain.select('slope'), buffered_point, 'slope'))


    # NDMI from Landsat 8
    landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
                .filterBounds(point) \
                .filterDate(date_start, date_end) \
                .median()
    ndmi_image = landsat.normalizedDifference(['SR_B5', 'SR_B6']).rename('NDMI')
    ndmis.append(extract_mean_value(ndmi_image, buffered_point, 'NDMI'))


# Attach results to DataFrame
data['elevation'] = elevations
data['landcover'] = landcovers
data['aspect'] = aspects
data['slope'] = slopes
data['ndmi'] = ndmis

# Export
data.to_csv("WildFire_DataSet.csv", index=False)
print("Data enrichment completed and saved.")